In [1]:
from pytube import YouTube, Channel
import whisper
import os
os.environ["IMAGEIO_FFMPEG_EXE"] = "/usr/bin/ffmpeg"
from moviepy.editor import AudioFileClip
from googleapiclient.discovery import build
from datetime import datetime, timedelta
from dotenv import load_dotenv
from transformers import AutoTokenizer 
from transformers import AutoModelForSequenceClassification
from transformers import pipeline  

In [ ]:
#This is where the code starts

In [2]:
from pytube import YouTube, Channel
import whisper
import os
os.environ["IMAGEIO_FFMPEG_EXE"] = "/usr/bin/ffmpeg"
from moviepy.editor import AudioFileClip
from googleapiclient.discovery import build
from datetime import datetime, timedelta
from dotenv import load_dotenv
from transformers import AutoTokenizer 
from transformers import AutoModelForSequenceClassification
from transformers import pipeline  

In [28]:
#### This is the function for sentimen analysis and it works!
#This cell below is the most important of all. 
# Please send me the output file that gets saved so I see the columns
## if this cell below works, don't worry about anything else 

In [3]:
# This is how we get our function called final_df
import pandas as pd
import openai
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Initialize OpenAI API
openai.api_key = 'sk-proj-ggeheV85h9E9BHTgpycNT3BlbkFJIANFULv09YXo8exqduw0'

#  selected Huggingface models with their corresponding paths. We pick FinancialBert and ProsusAI/finbert
models = {
    "FinancialBERT": "yiyanghkust/finbert-tone",
    #"bert-base-uncased": "bert-base-uncased",
    "ProsusAI/finbert": "ProsusAI/finbert"
}

def analyze_sentiment_gpt35turbo(text):
    """
    Analyze sentiment of a given text using GPT-3.5-turbo.

    Parameters:
    - text (str): The input text to analyze.

    Returns:
    - sentiment (str): The sentiment label of the text.
    """
    if not text:
        return "We need text"  

    messages = [
        {"role": "system", "content": "You are a sentiment analysis assistant."},
        {"role": "user", "content": f"Please analyze the sentiment of the following text and classify it as Positive, Negative, or Neutral:\n\n{text}"}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    sentiment = response.choices[0].message['content'].strip()
    return sentiment

def analyze_sentiment_huggingface(text, model_path):
    """
    Analyze sentiment of a given text using a specified pre-trained model from Hugging Face.

    Parameters:
    - text (str): The input text to analyze.
    - model_path (str): The path to the pre-trained model.

    Returns:
    - label (str): The sentiment label of the text.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    sentiment_analyzer = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
    
    result = sentiment_analyzer(text)[0]
    return result['label']

def analyze_csv_sentiments(input_csv, output_csv):
    """
    Analyze sentiments of text data in a CSV file using multiple pre-trained models and
    store the results in separate columns for each model.

    Parameters:
    - input_csv (str): Path to the input CSV file containing text data.
    - output_csv (str): Path to save the output CSV file with sentiment analysis results.

    Returns:
    - df (DataFrame): The updated DataFrame with sentiment analysis results.
    """
    # Load the CSV file into a DataFrame
    df = pd.read_csv(input_csv)

    # Ensure there is a column to analyze
    if 'text' not in df.columns:
        raise ValueError("The CSV file must contain a 'text' column for analysis.")

    # Analyze sentiment for each row using each model and add results to new columns
    df['gpt-3.5-turbo_sentiment'] = df['text'].apply(analyze_sentiment_gpt35turbo)
    
    for model_name, model_path in models.items():
        df[f'{model_name}_sentiment'] = df['text'].apply(lambda text: analyze_sentiment_huggingface(text, model_path))

    # Save the updated DataFrame to a new CSV file
    df.to_csv(output_csv, index=False)
    
    # Return the updated DataFrame
    return df




In [7]:
# This will take a while and this is how we generate our file with the above function
import warnings
warnings.filterwarnings("ignore")
input_csv = "test_file.csv"  # Path to your input CSV file
output_csv = "output_with_sentiments1.csv"  # Path to save the output CSV file with sentiments
result_df = analyze_csv_sentiments(input_csv, output_csv)
print(result_df.head())  # Print the first few rows of the updated DataFrame

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are

   Unnamed: 0                                               text  \
0           0  According to Gran , the company has no plans t...   
1           1  Technopolis plans to develop in stages an area...   
2           2  The international electronic industry company ...   
3           3  With the new production plant the company woul...   
4           4  According to the company 's updated strategy f...   

                             gpt-3.5-turbo_sentiment FinancialBERT_sentiment  \
0                                            Neutral                 Neutral   
1             The sentiment of the text is Positive.                 Neutral   
2  The sentiment of the text is Negative, as it m...                 Neutral   
3             The sentiment of the text is Positive.                Positive   
4                                            Neutral                Positive   

  bert-base-uncased_sentiment ProsusAI/finbert_sentiment  
0                     LABEL_0                    ne

In [21]:
# This is for simplifying the answer of gpt 3.5 turboso we only get the positive, neutral and negative from the 
# entire sentiment sentence. 

def find_sentiment_words(input_csv, output_csv, column_name):
    df = pd.read_csv(input_csv)
    
    if column_name not in df.columns:
        raise ValueError(f"The column {column_name} does not in the CSV file.")
        
    sentiment_words = ["Positive","Neutral","Negative"]
    
    df['chatgpt_sentiment'] = None
    
    for index,row in df.iterrows():
        text = row[column_name]
        
        for word in sentiment_words:
            if pd.notnull(text) and word.lower() in text.lower():
                df.at[index, 'chatgpt_sentiment'] = word
                break
    
    df.to_csv(output_csv, index = False)
    
    return df

In [22]:
input_csv = "output_with_sentiments1.csv"
output_csv = "updated_sentiments.csv"
column_name = "gpt-3.5-turbo_sentiment"


In [23]:
# updated version of our file with chatgpt_sentiment column added 
final_df = find_sentiment_words(input_csv,output_csv, column_name)
final_df

,Unnamed: 0,text,gpt-3.5-turbo_sentiment,FinancialBERT_sentiment,bert-base-uncased_sentiment,ProsusAI/finbert_sentiment,chatgpt_sentiment
0,0,"According to Gran , the company has no plans t...",Neutral,Neutral,LABEL_0,neutral,Neutral
1,1,Technopolis plans to develop in stages an area...,The sentiment of the text is Positive.,Neutral,LABEL_1,neutral,Positive
2,2,The international electronic industry company ...,"The sentiment of the text is Negative, as it m...",Neutral,LABEL_0,negative,Negative
3,3,With the new production plant the company woul...,The sentiment of the text is Positive.,Positive,LABEL_1,positive,Positive
4,4,According to the company 's updated strategy f...,Neutral,Positive,LABEL_0,positive,Neutral
...,...,...,...,...,...,...,...
4841,4841,LONDON MarketWatch -- Share prices ended lower...,The sentiment of the text is Negative.,Negative,LABEL_1,negative,Negative
4842,4842,Rinkuskiai 's beer sales fell by 6.5 per cent ...,This text contains a mix of positive and negat...,Positive,LABEL_0,negative,Positive
4843,4843,Operating profit fell to EUR 35.4 mn from EUR ...,The sentiment of the text is Neutral.,Negative,LABEL_1,negative,Neutral
4844,4844,Net sales of the Paper segment decreased to EU...,This text contains a factual statement regardi...,Negative,LABEL_1,negative,Neutral


In [24]:
# # This is where we want to change the lable to positive (we dropped this column later today)
# final_df[final_df['bert-base-uncased_sentiment'] == 'LABEL_1'] = "Positive"

In [26]:
# Again just some processing on the columns like doing upper case so all match. 

        
        #print(final_df)
        
replacements = {"positive":"Positive", 
                "neutral":"Neutral",
               "negative":"Negative"
    }
for old_word, new_word in replacements.items(): 
        final_df['FinancialBERT_sentiment'].replace(old_word, new_word, inplace=True)
        
        # print(final_df)        
replacements = {"positive":"Positive", 
                "neutral":"Neutral",
               "negative":"Negative"
    }
for old_word, new_word in replacements.items(): 
        final_df['ProsusAI/finbert_sentiment'].replace(old_word, new_word, inplace=True)
        
       # print(final_df)

     Unnamed: 0                                               text  \
0             0  According to Gran , the company has no plans t...   
1      Positive                                           Positive   
2             2  The international electronic industry company ...   
3      Positive                                           Positive   
4             4  According to the company 's updated strategy f...   
...         ...                                                ...   
4841   Positive                                           Positive   
4842       4842  Rinkuskiai 's beer sales fell by 6.5 per cent ...   
4843   Positive                                           Positive   
4844   Positive                                           Positive   
4845   Positive                                           Positive   

                                gpt-3.5-turbo_sentiment  \
0                                               Neutral   
1                                        

In [ ]:
# (4846, 6)
final_df.shape

In [ ]:
# saving the file that we have the sentiment model results in, call it "test_models"

final_df.to_csv('test_models.csv')

### we now need to check these sentiment labels against the Kaggle dataframe: 

In [ ]:
# Kaggle datafram is stored in test: 

test_file = "/Users/nazanin.komeilizadeh/Desktop/test.csv"
test = pd.read_csv(test_file)

In [ ]:
replacements = {"positive":"Positive", 
                "neutral":"Neutral",
               "negative":"Negative"
    }
for old_word, new_word in replacements.items(): 
        test['kaggle_label'].replace(old_word, new_word, inplace=True)
        

In [ ]:
test.kaggle_label.unique()

In [ ]:
test = test.drop(['all_data'], axis = 1)

In [ ]:
# save into a csv file 
test.to_csv('kaggle.csv')

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score

# here, we want to check the sentiment analysis of each model against the Kaggle label 

# columns to compare
columns_final_df = ['FinancialBERT_sentiment','ProsusAI/finbert_sentiment', 'chatgpt_sentiment']
column_test = 'kaggle_label'

# making sure DataFrames have the same length for row-wise comparison
if len(final_df) != len(test):
    raise ValueError("DataFrames must have the same number of rows for comparison.")

# Create new columns in final_df to store the match results
for col in columns_final_df:
    final_df[f'{col}_match'] = final_df[col] == test[column_test]

In [ ]:
final_df.to_csv('match.csv')

In [ ]:
# slicing the datafram to only the last 3 comparision columns

accuracy = final_df[final_df.columns[-3:]]


In [ ]:
accuracy.columns


In [ ]:
accuracy['FinancialBERT_sentiment_match'].value_counts()


In [ ]:
accuracy['ProsusAI/finbert_sentiment_match'].value_counts()


In [ ]:
accuracy['chatgpt_sentiment_match'].value_counts()


In [1]:
len(accuracy[accuracy['FinancialBERT_sentiment_match']==1])/len(accuracy)
# 0.79

NameError: name 'accuracy' is not defined

In [ ]:
# turns out it's the best model for accuracy
len(accuracy[accuracy['ProsusAI/finbert_sentiment_match']==True])/len(accuracy)
# 0.8895

In [ ]:
len(accuracy[accuracy['chatgpt_sentiment_match']==True])/len(accuracy)
# 0.712